In [6]:

from monai.utils import first, set_determinism
from monai.transforms import (EnsureChannelFirstd, Compose, CropForegroundd, LoadImaged, Orientationd, RandCropByPosNegLabeld, ScaleIntensityRanged, Spacingd)
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset
from monai.apps import download_and_extract
from monai.transforms import CenterSpatialCropd
from monai.transforms import Resized
import torch
import matplotlib.pyplot as plt
import os
import glob
import torch.nn as nn
import json
from datetime import datetime
from data_preparation2 import DataHandling 
from UNet_model import create_unet
import numpy as np
import nibabel as nib
from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd, Spacingd,
    SpatialPadd, ScaleIntensityd, CenterSpatialCropd
)

import math


In [7]:
max_epochs = 500
starting_epoch = 0
decay_epoch = 5
learning_rate = 0.001

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=1,
    channels=(32, 64, 128, 256),

    act=(nn.ReLU, {"inplace": True}),
    strides=(2, 2, 2, 2),
  num_res_units=2,
    norm=Norm.BATCH,
).to(device)

loss_function = torch.nn.MSELoss()

print('Defining optimizer...')
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.5, 0.999))




Defining optimizer...


/homes/zshahpouri/DLP/.venv/lib/python3.11/site-packages/monai/networks/nets/unet.py:133: UserWarning: `len(strides) > len(channels) - 1`, the last 1 values of strides will not be used.
  warnings.warn(f"`len(strides) > len(channels) - 1`, the last {delta} values of strides will not be used.")


-------------
# DecayLR

In [8]:
import torch


class DecayLR:
    def __init__(self, epochs, offset, decay_epochs):
        epoch_flag = epochs - decay_epochs
        assert (epoch_flag > 0), "Decay must start before the training session ends!"
        self.epochs = epochs
        self.offset = offset
        self.decay_epochs = decay_epochs

    def step(self, epoch):
        return 1.0 - max(0, epoch + self.offset - self.decay_epochs) / (
                self.epochs - self.decay_epochs)
    


In [9]:
import torch

# Assuming the DecayLR class, model definition, optimizer, and scheduler are defined as in your snippet.



lr_lambda = DecayLR(epochs=max_epochs, offset=0, decay_epochs=decay_epoch).step
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)

print("Initial learning rate:", optimizer.param_groups[0]['lr'])

for epoch in range(max_epochs):
    # Simulate the scheduler's step at each epoch
    scheduler.step(epoch)
    
    # Access the current learning rate
    current_lr = scheduler.get_last_lr()[0]  # get_last_lr() is recommended for accessing the last computed LR
    print(f"Epoch {epoch+1}/{max_epochs}, Current Lr: {current_lr}")


Initial learning rate: 0.001
Epoch 1/500, Current Lr: 0.001
Epoch 2/500, Current Lr: 0.001
Epoch 3/500, Current Lr: 0.001
Epoch 4/500, Current Lr: 0.001
Epoch 5/500, Current Lr: 0.001
Epoch 6/500, Current Lr: 0.001
Epoch 7/500, Current Lr: 0.000997979797979798
Epoch 8/500, Current Lr: 0.000995959595959596
Epoch 9/500, Current Lr: 0.000993939393939394
Epoch 10/500, Current Lr: 0.0009919191919191919
Epoch 11/500, Current Lr: 0.00098989898989899
Epoch 12/500, Current Lr: 0.000987878787878788
Epoch 13/500, Current Lr: 0.0009858585858585859
Epoch 14/500, Current Lr: 0.000983838383838384
Epoch 15/500, Current Lr: 0.0009818181818181818
Epoch 16/500, Current Lr: 0.0009797979797979799
Epoch 17/500, Current Lr: 0.0009777777777777777
Epoch 18/500, Current Lr: 0.0009757575757575757
Epoch 19/500, Current Lr: 0.0009737373737373739
Epoch 20/500, Current Lr: 0.0009717171717171718
Epoch 21/500, Current Lr: 0.0009696969696969698
Epoch 22/500, Current Lr: 0.0009676767676767677
Epoch 23/500, Current Lr: 0

-------------------------
# CosineAnnealingLR

In [11]:
import torch

# Assuming 'model' is defined
max_epochs = 300  # Example: total number of epochs
learning_rate = 1e-3  # Starting learning rate

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max_epochs)

print("Initial learning rate:", optimizer.param_groups[0]['lr'])

for epoch in range(max_epochs):
    optimizer.step()  # Dummy optimizer step, replace with actual training step
    scheduler.step()
    
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Epoch {epoch+1}/{max_epochs}, Current LR: {current_lr}")

    


Initial learning rate: 0.001
Epoch 1/300, Current LR: 0.000999972584682756
Epoch 2/300, Current LR: 0.0009998903417374227
Epoch 3/300, Current LR: 0.0009997532801828658
Epoch 4/300, Current LR: 0.0009995614150494292
Epoch 5/300, Current LR: 0.000999314767377287
Epoch 6/300, Current LR: 0.0009990133642141358
Epoch 7/300, Current LR: 0.000998657238612229
Epoch 8/300, Current LR: 0.0009982464296247522
Epoch 9/300, Current LR: 0.00099778098230154
Epoch 10/300, Current LR: 0.0009972609476841367
Epoch 11/300, Current LR: 0.0009966863828001983
Epoch 12/300, Current LR: 0.0009960573506572392
Epoch 13/300, Current LR: 0.000995373920235722
Epoch 14/300, Current LR: 0.0009946361664814943
Epoch 15/300, Current LR: 0.000993844170297569
Epoch 16/300, Current LR: 0.0009929980185352527
Epoch 17/300, Current LR: 0.000992097803984621
Epoch 18/300, Current LR: 0.0009911436253643446
Epoch 19/300, Current LR: 0.0009901355873108612
Epoch 20/300, Current LR: 0.000989073800366903
Epoch 21/300, Current LR: 0.0

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)

optimizer = torch.optim.Adam(model.parameters(), 1e-5)



In [ ]:
lr=0.0001 # type=float, help="learning rate"
lr_step_size=40 # type=int, help="decay learning rate every lr_step_size epochs")
lr_gamma =0.1 # type=float, help="every lr_step_size epochs, decay learning rate by a factor of lr_gamma",
weight_decay=0.0 # type=float, help="ridge regularization factor")


optimizer = torch.optim.Adam(model.parameters(), lr, weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, lr_step_size, lr_gamma)